## Rough-Level Alignment: Locate Subset in Open Subtitle

In the rough alignment phrase, we try to align each utterance in tbbt episode with a open subtitle index. Specifically, we do the following:

1.Clean strings and then perform matching
2.assign index to each utterance. So each utterance is assigned a set of subtitles [index, segment, en_subtitle, zh_subtitle]
--index is the subtitle's index in open subtitle dataset
--segment is the utterance segment for alignment
--en_subtitle and zh_subtitle is the whole subtitle in English and Chinese language.


In [1]:
import pickle as pkl
import json
from collections import defaultdict
import jiwer

In [2]:
"""
Organize index within one episode
Input: ([[]])
Output: {index: []}
"""
def get_index_dict(episode):
    index_dict = defaultdict()
    for idx, segment, en_sub, zh_sub in episode:
        temp = [segment, en_sub, zh_sub]
        if idx not in index_dict:
            index_dict[idx] = [temp]
        else:
            index_dict[idx].append(temp)
    return index_dict

In [3]:
# Organize data
def organize_by_seasons(all_data):
    res = {}
    for epi in list(all_data.keys()):
        season = int(epi[1:3])
        episode = int(epi[-2:])
        # Process the data in one episode
        temp = get_index_dict(all_data[epi])
        if season not in res:
            res[season] = {
                episode: temp
            }
        else:
            res[season][episode] = temp
    return res

In [4]:
"""
Get all the indexs within one episode
Input: {index: [[]]}
Output: sorted index list
"""
def get_epi_indexs_gaps(episode):
    idx_list = []
    for idx in episode:
        idx_list.append(idx)
    idx_list.sort()
    # Calculate gaps
    gaps = calculate_gaps(idx_list)
    return idx_list, gaps

In [5]:
"""
Calculate gaps between elements given an list of integer
"""
def calculate_gaps(idx_list):
    gaps = []
    idx_list.sort()
    for i in range(len(idx_list)-1):
        gaps.append(idx_list[i+1]-idx_list[i])
    return gaps

In [6]:
"""
Locate continuous subset that gap between to indexs is small than threshold
Input: indexs, gaps, threshold
Output: indexs of continuous subset
"""
def find_all_continuous_subsets(idx_list, gaps, len_threshold, gap_threshold):
    res = []
    path = [idx_list[0]]
    for i in range(len(gaps)):
        if gaps[i]<=gap_threshold:
            path.append(idx_list[i+1])
        else:
            if len(path)>=len_threshold:
                res.append(path)
            path = [idx_list[i+1]]
    return res

In [7]:
# Load search result
with open('indexs_friends.pkl', 'rb') as f:
    temp = pkl.load(f)
print(len(temp))

236


In [8]:
results = organize_by_seasons(temp)

In [12]:
# Check all substrings in each episode
temp = {}
for season_id in sorted(list(results.keys())):
    if season_id not in temp:
        temp[season_id] = []
    season = results[season_id]
    for episode_id in sorted(list(season.keys())):
        idx_list, gaps = get_epi_indexs_gaps(results[season_id][episode_id])
        subsets = find_all_continuous_subsets(idx_list, gaps, 6, 100)
        try:
            gaps = calculate_gaps(subsets[-1])
            print(gaps)
            print("Season:", season_id, "|Episode:", episode_id, "|Subset Length:", len(subsets[-1]), "|Sum:", sum(gaps), "|Maximum:", max(gaps))
            temp[season_id].append(episode_id)
        except:
            print("Season:", season_id, "Episode:", episode_id, "Subset Length:", subsets)
        print('=='*50)

[4, 2, 3, 1, 1, 2, 4, 1, 2, 3, 2, 3, 10, 3, 2, 2, 4, 3, 4, 2, 6, 6, 1, 1, 3, 4, 1, 3, 3, 1, 3, 2, 4, 4, 5, 1, 1, 2, 1, 4, 11, 1, 5, 12, 6, 13, 2, 1, 1, 2, 2, 5, 1, 4, 1, 3, 5, 1, 11, 1, 1, 8, 2, 1, 4, 1, 1, 4, 1, 1, 2, 4, 1, 2, 2, 1, 2, 3, 2, 5, 5, 10, 9, 2, 3, 1, 14, 1, 1, 2, 1, 2, 4, 3, 1, 1, 1, 6, 1, 9, 1, 3, 4, 1, 6, 2, 1, 2, 1, 3, 2, 1, 6, 14, 3, 2, 1, 3, 1, 11, 2, 5, 4, 5, 1, 7, 1, 6, 10]
Season: 1 |Episode: 1 |Subset Length: 130 |Sum: 436 |Maximum: 14
[4, 1, 2, 4, 2, 1, 3, 4, 4, 13, 6, 3, 1, 9, 1, 7, 4, 1, 1, 7, 1, 5, 4, 2, 2, 3, 7, 1, 3, 7, 2, 1, 1, 3, 1, 1, 3, 1, 1, 2, 1, 3, 2, 2, 2, 9, 1, 1, 8, 2, 1, 1, 1, 2, 2, 1, 13, 7, 7, 4, 29, 3, 11, 2, 5, 5, 9, 3, 6, 3, 5, 1, 14, 1]
Season: 1 |Episode: 2 |Subset Length: 75 |Sum: 296 |Maximum: 29
[3, 5, 10, 1, 6, 2, 9, 7, 15, 4, 3, 1, 2, 5, 1, 4, 1, 1, 8, 2, 8, 1, 1, 1, 4, 1, 1, 4, 7, 9, 7, 3, 1, 4, 3, 8, 1, 2, 6, 3, 7, 16, 15, 7, 1, 3, 1, 1, 4, 4, 5, 6, 4, 2, 2, 2, 2, 3, 4, 1, 8, 16, 1, 3, 2, 7, 8, 4, 1]
Season: 1 |Episode: 3 |Subset Le

In [10]:
print("Season", "\t", "Episode")
for x in temp:
    print(x, "\t \t", sorted(temp[x]))

Season 	 Episode
1 	 	 [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
2 	 	 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
3 	 	 [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25]
4 	 	 [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22]
5 	 	 [2, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24]
6 	 	 [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25]
7 	 	 [4, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23]
8 	 	 [3, 4, 12, 13, 15, 17, 19, 21, 22, 23, 24]
9 	 	 [2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18, 21]
10 	 	 [2, 5, 7, 9, 10, 11, 12, 15, 16, 17, 18]


## Fine-Level Alignment: Search within the Open Subtitle Subset

In [218]:
# Load Open Subtitle
with open('en_subtitles.pkl', 'rb') as f:
    en_subtitle = pkl.load(f)
with open('zh_subtitles.pkl', 'rb') as f:
    zh_subtitle = pkl.load(f)

In [395]:
# Load Memor Dataset
with open('memor/data.json') as f:
    tbbt = json.load(f)

In [400]:
# Define sentence transformation
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.ExpandCommonEnglishContractions(),
    jiwer.RemovePunctuation(),
    jiwer.Strip()
])

In [408]:
"""
Experiment within one season: Season 1, Episode 1
"""
idx_list, gaps = get_epi_indexs_gaps(results[1][1])
subsets = find_all_continuous_subsets(idx_list, gaps, 6, 100)[-1]
# Calculate gaps within the subset
gaps_subsets = calculate_gaps(subsets)

# Prepare Subtitle Subset
bias = 200
start = subsets[0]-bias
end = subsets[-1]+bias
en_subset = en_subtitle[start: end]
zh_subset = zh_subtitle[start: end]

# Prepare utterances of one episode
tbbt_episode = []
for item in tbbt:
    if item.strip().split('_')[0]=='S01E01':
        sentences = tbbt[item]['sentences']
        speakers = tbbt[item]['speakers']
        for sentence, speaker in zip(sentences, speakers):
            # if len(tbbt_episode)!=0:
            #     if sentence==tbbt_episode[-1][0]:
            #         print(sentence)
            #         print(tbbt_episode[-1])
            #         print("=="*50)
            tbbt_episode.append([sentence, speaker])

In [409]:
# Define sentence transformation
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.ExpandCommonEnglishContractions(),
    jiwer.RemovePunctuation(),
    jiwer.Strip()
])

In [539]:
"""
Expand from exisiting episode utterance
Iterate each episode utterance(that is already aligned)
Then, check whether the subtitle in min(epi_id)-1 or max(epi_id)+1 is within utterance
"""
def extend_from_epi_alignment(en_subset, epi2sub_alignment_2):
    for epi_id in epi2sub_alignment_2:
        # Define episode utterance id
        epi_id_former = epi_id-1
        epi_id = epi_id
        epi_id_latter = epi_id+1

        # Define subtitle id
        sub_id_former = min(epi2sub_alignment_2[epi_id])-1
        sub_id_latter = max(epi2sub_alignment_2[epi_id])+1

        # Check whether subtitle nearby is in the utterance
        sub_former = transformation(en_subset[sub_id_former])
        sub_latter = transformation(en_subset[sub_id_latter])
        epi = transformation(tbbt_episode[epi_id][0])
        if sub_former in epi:
            epi2sub_alignment_2[epi_id].append(sub_id_former)
        if sub_latter in epi:
            epi2sub_alignment_2[epi_id].append(sub_id_latter)

    return epi2sub_alignment_2

In [372]:
"""
Filter alignment by index gap
"""
def filter_alignment_by_gap(sub2epi):
    # Track the epi_id to filter the alignment
    track = []
    start = sub2epi[list(sub2epi.keys())[0]]
    for item in start:
        track.append(item)

    # Filter alignment by gap
    temp = {}
    for sub_id in sub2epi:
        for epi_id in sub2epi[sub_id]:
            if 0<= abs(epi_id-track[-1]) <= 10:
                track.append(epi_id)
                if sub_id not in temp:
                    temp[sub_id] = set()
                    temp[sub_id].add(epi_id)
                else:
                    temp[sub_id].add(epi_id)

    return temp

In [262]:
"""
Use cleaned open subtitle to match substring of utterance in episode
Each subtitle sentence is split into 6 tokens
"""
def string_match(en_subset, episode):
    res = {}
    for i, subtitle in enumerate(en_subset):
        subtitle = transformation(subtitle)
        subtitle_tokens = subtitle.strip().split(" ")
        if len(subtitle_tokens)<5:
            continue
        # Build subtitle segments
        subtitle_segments = []
        num_iter = len(subtitle_tokens) // 5
        for j in range(num_iter):
            subtitle_segments.append(" ".join(subtitle_tokens[j*5: j*5+5]))

        for j, (utt, speaker) in enumerate(tbbt_episode):
            utt = transformation(utt)
            for sub_seg in subtitle_segments:
                if sub_seg in utt:
                    if i not in res:
                        res[i] = set()
                        res[i].add(j)
                    else:
                        res[i].add(j)
    return res

In [282]:
alignment_string = string_match(en_subset, tbbt_episode)
print(alignment_string)
print(len(alignment_string))

{200: {0}, 202: {0}, 203: {0}, 204: {1}, 205: {2}, 206: {3}, 208: {3}, 210: {3}, 222: {4}, 223: {4}, 224: {4}, 225: {5}, 226: {5}, 227: {6}, 228: {6}, 229: {6}, 230: {9, 7}, 234: {14}, 235: {15}, 242: {16}, 246: {18}, 247: {18}, 252: {21}, 253: {22}, 256: {27}, 287: {37}, 288: {38}, 289: {40}, 290: {42}, 297: {43}, 300: {46}, 304: {47}, 305: {48}, 307: {50}, 308: {51}, 310: {52}, 314: {53}, 315: {53}, 316: {53}, 317: {53}, 318: {54}, 327: {63}, 329: {66}, 334: {68}, 336: {70}, 337: {71}, 338: {77, 71}, 339: {71}, 341: {72, 73}, 342: {74}, 343: {75}, 348: {75}, 351: {76}, 353: {77, 71}, 355: {77}, 358: {79}, 360: {82}, 361: {82}, 362: {83}, 363: {85}, 366: {87}, 368: {88}, 382: {89}, 393: {91}, 394: {92}, 395: {92}, 396: {93}, 398: {94}, 416: {95}, 417: {96}, 419: {98}, 424: {99}, 427: {100, 101}, 428: {100, 101}, 429: {100, 101}, 430: {102}, 431: {103}, 433: {105}, 434: {105}, 435: {106}, 439: {107}, 442: {109}, 453: {113}, 455: {113}, 456: {113}, 457: {114}, 461: {115}, 462: {116}, 46

In [494]:
"""
Use cleaned open subtitle to match substring of utterance in episode
Each subtitle sentence is split into 6 tokens
"""
def string_match_sliding_window(en_subset, episode, window_size):
    res = {}
    for i, subtitle in enumerate(en_subset):
        subtitle = transformation(subtitle)
        # for j, (utt, speaker) in enumerate(tbbt_episode):
        #     utt = transformation(utt)
        #     if subtitle==utt:
        #         if i not in res:
        #             res[i] = set()
        #             res[i].add(j)
        #         else:
        #             res[i].add(j)

        subtitle = transformation(subtitle)
        subtitle_tokens = subtitle.strip().split(" ")
        if len(subtitle_tokens)<window_size:
            continue
        # # Build subtitle segments
        # subtitle_segments = []
        # num_iter = len(subtitle_tokens) // 6
        # for j in range(num_iter):
        #     subtitle_segments.append(" ".join(subtitle_tokens[j*6: j*6+6])
        # Build subtitle segments using sliding algorithm
        # print("Subtitle Segments:")
        subtitle_segments = []
        for j in range(len(subtitle_tokens)-window_size):
            subtitle_segments.append(" ".join(subtitle_tokens[j: j+window_size]))
            # print(" ".join(subtitle_tokens[j: j+5]))

        for j, (utt, speaker) in enumerate(tbbt_episode):
            utt = transformation(utt)
            for sub_seg in subtitle_segments:
                if sub_seg in utt:
                    if i not in res:
                        res[i] = set()
                        res[i].add(j)
                    else:
                        res[i].add(j)
    temp = filter_alignment_by_gap(res)

    return temp

In [489]:
alignment_string_slide = string_match_sliding_window(en_subset, tbbt_episode, window_size=4)
print(alignment_string_slide)
print(len(alignment_string_slide))

{164: {55}, 299: {45}, 300: {46}, 304: {47}, 305: {48}, 306: {49}, 307: {50}, 308: {51}, 310: {52}, 314: {53}, 315: {53}, 316: {53}, 317: {53}, 318: {54}, 319: {56}, 320: {59}, 322: {63}, 327: {63}, 329: {66}, 330: {67}, 334: {68}, 335: {69}, 336: {70}, 337: {71}, 338: {77, 71}, 339: {71}, 340: {71}, 341: {72, 73}, 342: {74}, 343: {75}, 348: {75}, 351: {76}, 353: {77, 71}, 355: {77}, 356: {78}, 358: {79}, 360: {82}, 361: {82}, 362: {83}, 363: {85}, 365: {86}, 366: {87}, 367: {88}, 368: {88}, 382: {89}, 383: {90}, 393: {91}, 394: {92}, 395: {92}, 396: {93}, 398: {94}, 416: {95}, 417: {96}, 418: {97}, 419: {98}, 424: {99}, 427: {100, 101}, 428: {100, 101}, 429: {100, 101}, 430: {102}, 431: {103}, 433: {105}, 434: {105}, 435: {106}, 439: {107}, 442: {109}, 444: {110}, 453: {113}, 455: {113}, 456: {113}, 457: {114}, 461: {115}, 462: {116}, 463: {116}, 464: {117}, 467: {118}, 468: {119}, 469: {119}, 470: {119}, 471: {119}, 477: {120}, 478: {121}, 479: {123}, 493: {125}, 506: {126}, 522: {12

In [285]:
"""
Calculate Edit Distance between subtitle and episode
把subtitle根据标点符号分成几个小部分
把episode也根据标点符号分成几个小部分
然后对subtitle对不同组合，以及episode对不同组合
然后用wer来算edit distance
"""
def get_optimal_wer_from_episode(ground_truth, hypothesis_pool):
    scores = []
    for i, hypothesis in enumerate(hypothesis_pool):
        scores.append(jiwer.compute_measures(ground_truth, hypothesis)['wer'])
    return min(scores), hypothesis_pool[scores.index(min(scores))], ground_truth, scores.index(min(scores))


def wer_match(en_subset, episode):
    res = {}
    print(en_subset)

    for i, subtitle in enumerate(en_subset):
        # print("Whole Subtitle:", subtitle)
        subtitle = transformation(subtitle)
        subtitle_tokens = subtitle.strip().split(" ")
        if len(subtitle_tokens)<5:
            continue

        subtitle_segments = []
        for j in range(len(subtitle_tokens)-5):
            subtitle_segments.append(" ".join(subtitle_tokens[j: j+5]))
            print(" ".join(subtitle_tokens[j: j+5]))

        for j, (utt, speaker) in enumerate(tbbt_episode):
            utt = transformation(utt)
            for sub_seg in subtitle_segments:
                if sub_seg in utt:
                    if i not in res:
                        res[i] = set()
                        res[i].add(j)
                    else:
                        res[i].add(j)

    return res

In [286]:
alignment_wer = wer_match(en_subset, tbbt_episode)

['Thank you very much. Good day to you.', 'Good day to you.', 'Come and buy a dresser!', 'The years with Iisakki passed quickly.', 'Before I knew it, I was all grown up, with a beard and all.', 'The village had grown.', 'There were so many new children - that me and Iisakki could not keep count.', 'But we had a secret helper.', 'Nikolas.', '-Eemeli.', 'Long time no see. You should come more often.', "I've been busy. Iisakki is no longer young.", 'Do you have the list?', "Well, I'll be... So many new children.", 'As a matter of fact, one name is missing from that list. Elsa?', 'Is that...', '-A girl, three months.', "Let's add her to the list.", 'What is the name of this little princess?', 'Aada.', 'Aada?', 'Hello, Aada.', 'Nikolas, meet Henrik and Hermanni.', 'My sons.', 'I sought them out and asked them here.', 'We were wrong when we...', 'We want to make it up to our father.', 'We came to take him to live with us.', '-To live with you? Where?', 'Away from here. Father is too old to b

In [ ]:
"""
Extend
"""

In [513]:
# Step 1: Alignment
sub2epi_alignment_2 = string_match_sliding_window(en_subset, tbbt_episode, window_size=6)

# Turn sub2epi into epi2sub
epi2sub_alignment_2 = {}
for x in sub2epi_alignment_2:
    if len(sub2epi_alignment_2[x])!=1:
        continue
    sub_id = x
    epi_id = list(sub2epi_alignment_2[x])[0]
    # print(epi_id, sub_id)

    if epi_id not in epi2sub_alignment_2:
        epi2sub_alignment_2[epi_id] = [sub_id]
    else:
        epi2sub_alignment_2[epi_id].append(sub_id)

# Full-fill the gap
for x in epi2sub_alignment_2:
    epi2sub_alignment_2[x] = list(range(min(epi2sub_alignment_2[x]), max(epi2sub_alignment_2[x])+1))

# Get all indexs
all_epi = []
all_sub = []
for x in epi2sub_alignment_2:
    all_epi.append(x)
    all_sub.extend(epi2sub_alignment_2[x])

In [537]:
epi2sub_alignment_2 = extend_from_epi_alignment(en_subset, epi2sub_alignment_2)

In [538]:
print(epi2sub_alignment_2)

{0: [200, 201, 202, 203], 2: [205], 3: [206, 207, 208, 209, 210, 211], 4: [223, 224, 222], 5: [226, 225], 6: [227, 228, 229], 14: [234], 15: [235], 16: [242], 18: [246, 245, 247], 22: [253], 27: [256], 29: [263], 37: [287], 38: [288], 40: [289], 42: [290], 43: [297], 46: [300], 47: [304], 48: [305], 50: [307], 51: [308], 53: [314, 315, 316, 317], 63: [327], 68: [334], 70: [336], 71: [337, 338, 339, 340], 74: [342], 75: [343], 76: [351], 79: [358, 357], 82: [360, 361], 83: [362], 85: [363], 87: [366], 88: [368, 369], 91: [393], 92: [394, 395], 93: [396], 94: [398], 95: [416, 415], 96: [417], 98: [419], 99: [424, 425], 102: [430], 103: [431], 105: [434, 433], 107: [439, 440], 113: [453, 454, 455, 456, 452], 114: [457], 115: [461], 116: [462, 463], 117: [464], 118: [467], 119: [470, 469, 468], 120: [477], 121: [478], 125: [493], 129: [522, 523, 524], 139: [531], 140: [532], 142: [536, 537, 538], 144: [541, 540]}


In [496]:
for epi_id in epi2sub_alignment_2:
    print(epi_id, epi2sub_alignment_2[epi_id])

0 [200, 201, 202, 203]
2 [205]
3 [206, 207, 208, 209, 210]
4 [223, 224]
5 [226]
6 [227, 228, 229]
14 [234]
15 [235]
16 [242]
18 [246]
22 [253]
27 [256]
29 [263]
37 [287]
38 [288]
40 [289]
42 [290]
43 [297]
46 [300]
47 [304]
48 [305]
50 [307]
51 [308]
53 [314, 315, 316, 317]
63 [327]
68 [334]
70 [336]
71 [337, 338, 339, 340]
74 [342]
75 [343]
76 [351]
79 [358]
82 [360, 361]
83 [362]
85 [363]
87 [366]
88 [368]
91 [393]
92 [394, 395]
93 [396]
94 [398]
95 [416]
96 [417]
98 [419]
99 [424]
102 [430]
103 [431]
105 [434]
107 [439]
113 [453, 454, 455, 456]
114 [457]
115 [461]
116 [462, 463]
117 [464]
118 [467]
119 [470]
120 [477]
121 [478]
125 [493]
129 [522, 523, 524]
139 [531]
140 [532]
142 [536, 537, 538]
144 [541]


In [492]:
# Check the token length
for x in epi2sub_alignment_2:
    epi_tokens = transformation(tbbt_episode[x][0]).strip().split(" ")
    sub_tokens = transformation(" ".join([en_subset[sub_id] for sub_id in epi2sub_alignment_2[x]])).strip().split(" ")
    print(x, epi2sub_alignment_2[x])
    print(epi_tokens)
    print(sub_tokens)
    print(len(epi_tokens), len(sub_tokens))
    print("=="*50)

55 [164]
['well']
['well']
1 1
45 [299]
['this', 'is', 'really', 'impressive']
['this', 'is', 'really', 'impressive']
4 4
46 [300]
['i', 'have', 'a', 'board', 'if', 'you', 'like', 'boards', 'this', 'is', 'my', 'board']
['i', 'have', 'a', 'board', 'if', 'you', 'like', 'boards', 'this', 'is', 'my', 'board']
12 12
47 [304]
['at', 'least', 'i', 'did', 'not', 'have', 'to', 'invent', 'twentysix', 'dimensions', 'just', 'to', 'make', 'the', 'math', 'come', 'out']
['at', 'least', 'i', 'did', 'not', 'have', 'to', 'invent', '26', 'dimensions', 'just', 'to', 'make', 'the', 'math', 'come', 'out']
17 17
48 [305]
['i', 'did', 'not', 'invent', 'them', 'they', 'are', 'there']
['i', 'did', 'not', 'invent', 'them', 'they', 'are', 'there']
8 8
49 [306]
['in', 'what', 'universe']
['in', 'what', 'universe']
3 3
50 [307]
['in', 'all', 'of', 'them', 'that', 'is', 'the', 'point']
['in', 'all', 'of', 'them', 'that', 'is', 'the', 'point']
8 8
51 [308]
['uh', 'do', 'you', 'guys', 'mind', 'if', 'i', 'start']
['do'

In [493]:
# Print the TBBT Episode
for i, (utt, speaker) in enumerate(tbbt_episode):
    if i in all_epi:
        print(i, "||||", utt)
    else:
        print(i, utt)

0 if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. if it's unobserved it will, however, if it's observed after it's left the plane but before it hits its target, it will not have gone through both slits.
1 agreed, what's your point?
2 there's no point, i just think it's a good idea for a t-shirt.
3 one across is aegean, eight down is nabakov, twenty-six across is mcm, fourteen down is... move your finger... phylum, which makes fourteen across port-au-prince. see, papa doc's capital idea, that's port-au-prince. haiti.
4 no. we are committing genetic fraud. there's no guarantee that our sperm is going to generate high iq offspring, think about that. i have a sister with the same basic dna mix who hostesses at fuddruckers.
5 sheldon, this was your idea. a little extra money to get fractional t1 bandwidth in the apartment.
6 i know, and i do yearn for faster downloads, but there's some poor woman is going to pin her

In [452]:
# Print Open Subtitle
for i, subtitle in enumerate(en_subset):
    if i in all_sub:
        print(i, "||||", subtitle)
    else:
        print(i, subtitle)

0 Thank you very much. Good day to you.
1 Good day to you.
2 Come and buy a dresser!
3 The years with Iisakki passed quickly.
4 Before I knew it, I was all grown up, with a beard and all.
5 The village had grown.
6 There were so many new children - that me and Iisakki could not keep count.
7 But we had a secret helper.
8 Nikolas.
9 -Eemeli.
10 Long time no see. You should come more often.
11 I've been busy. Iisakki is no longer young.
12 Do you have the list?
13 Well, I'll be... So many new children.
14 As a matter of fact, one name is missing from that list. Elsa?
15 Is that...
16 -A girl, three months.
17 Let's add her to the list.
18 What is the name of this little princess?
19 Aada.
20 Aada?
21 Hello, Aada.
22 Nikolas, meet Henrik and Hermanni.
23 My sons.
24 I sought them out and asked them here.
25 We were wrong when we...
26 We want to make it up to our father.
27 We came to take him to live with us.
28 -To live with you? Where?
29 Away from here. Father is too old to be living 

In [449]:
print(sub2epi_alignment_2.keys())

dict_keys([200, 202, 203, 205, 206, 210, 223, 224, 226, 227, 229, 230, 234, 235, 242, 246, 253, 256, 263, 287, 288, 289, 290, 297, 300, 304, 305, 307, 308, 314, 316, 317, 327, 334, 336, 337, 340, 341, 342, 343, 351, 358, 360, 361, 362, 363, 366, 368, 393, 394, 395, 396, 398, 416, 417, 419, 424, 427, 428, 429, 430, 431, 434, 439, 453, 455, 456, 457, 461, 462, 463, 464, 467, 470, 477, 478, 493, 522, 524, 527, 531, 532, 536, 538, 541])


In [201]:
"""
Use cleaned open subtitle to match substring of utterance in episode
Each subtitle sentence is split into 6 tokens
"""
def string_wer(en_subset, episode):
    res = {}
    # for i, (utt, speaker) in enumerate(tbbt_episode):
    #     print(utt, speaker)

    for i, subtitle in enumerate(en_subset):
        if i not in alignment:
            print(subtitle)


        # subtitle = transformation(subtitle)
        # subtitle_tokens = subtitle.strip().split(" ")
        # if len(subtitle_tokens)<6:
        #     continue
        # # Build subtitle segments
        # subtitle_segments = []
        # num_iter = len(subtitle_tokens) // 6
        # for j in range(num_iter):
        #     subtitle_segments.append(" ".join(subtitle_tokens[j*6: j*6+6]))
        #
        # for j, (utt, speaker) in enumerate(tbbt_episode):
        #     utt = transformation(utt)
        #     for sub_seg in subtitle_segments:
        #         if sub_seg in utt:
        #             if i not in res:
        #                 res[i] = [j]
        #             else:
        #                 res[i].append(j)
    return res

In [202]:
temp = string_wer(en_subset, tbbt_episode)

Thank you very much. Good day to you.
Good day to you.
Come and buy a dresser!
The years with Iisakki passed quickly.
Before I knew it, I was all grown up, with a beard and all.
The village had grown.
There were so many new children - that me and Iisakki could not keep count.
But we had a secret helper.
Nikolas.
-Eemeli.
Long time no see. You should come more often.
I've been busy. Iisakki is no longer young.
Do you have the list?
Well, I'll be... So many new children.
As a matter of fact, one name is missing from that list. Elsa?
Is that...
-A girl, three months.
Let's add her to the list.
What is the name of this little princess?
Aada.
Aada?
Hello, Aada.
Nikolas, meet Henrik and Hermanni.
My sons.
I sought them out and asked them here.
We were wrong when we...
We want to make it up to our father.
We came to take him to live with us.
-To live with you? Where?
Away from here. Father is too old to be living in arctic conditions.
You'll get the house and the workshop. You've earned them.

In [204]:
print(transformation("- Agreed. What's your point?"))
print(transformation("agreed, what's your point?"))

agreed what is your point
agreed what is your point


In [205]:
print(transformation("Twenty-six across is MCM."))
print(transformation(" twenty-six across is mcm,"))

twentysix across is mcm
twentysix across is mcm


In [141]:
# Step 1: If subtitle is subset of the utterance
for subtitle in en_subset:
    subtitle = transformation(subtitle)
    for utt, speaker in tbbt_episode:
        utt = transformation(utt)
        if subtitle in utt:
            print("Subtitle:", subtitle)
            print("Utterance:", utt)
            print("=="*50)

Subtitle: is that
Utterance: i think what sheldon is trying to say is that sagittarius would not have been our first guess
Subtitle: is that
Utterance: you want to know the most pathetic part even though i hate his lying cheating guts i still love him is that crazy
Subtitle: red
Utterance: two hundred pound transvestite with a skin condition yes she is
Subtitle: red
Utterance: two hundred pound transvestite with a skin condition yes she is
Subtitle: i will show you
Utterance: come on i will show you the trick with the shower
Subtitle: that is nice
Utterance: oh that is nice
Subtitle: well
Utterance: well what do you want to do
Subtitle: well
Utterance: great well bye
Subtitle: well
Utterance: well
Subtitle: well
Utterance: well that is interesting leonard can not process corn
Subtitle: well
Utterance: well that is interesting leonard can not process corn
Subtitle: well
Utterance: well if that was a movie i would go see it
Subtitle: well
Utterance: well it sounds wonderful
Subtitle: wel

In [144]:
for utt, speaker in tbbt_episode:
    utt = transformation(utt)
    for subtitle in en_subset:
        if len(subtitle.strip().split(" "))<6:
            continue
        subtitle = transformation(subtitle)
        if subtitle in utt:
            print("Utterance:", utt)
            print("Subtitle:", subtitle)
            print("=="*50)

Utterance: if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits if it is unobserved it will however if it is observed after it is left the plane but before it hits its target it will not have gone through both slits
Subtitle: it will not have gone through both slits
Utterance: there is no point i just think it is a good idea for a tshirt
Subtitle: there is no point i just think it is a good idea for a tshirt
Utterance: one across is aegean eight down is nabakov twentysix across is mcm fourteen down is move your finger phylum which makes fourteen across portauprince see papa doc is capital idea that is portauprince haiti
Subtitle: fourteen down is move your finger
Utterance: one across is aegean eight down is nabakov twentysix across is mcm fourteen down is move your finger phylum which makes fourteen across portauprince see papa doc is capital idea that is portauprince haiti
Subtitle: see papa doc is capital idea tha

In [142]:
for subtitle in en_subset:
    subtitle = transformation(subtitle)
    print(subtitle)

thank you very much good day to you
good day to you
come and buy a dresser
the years with iisakki passed quickly
before i knew it i was all grown up with a beard and all
the village had grown
there were so many new children  that me and iisakki could not keep count
but we had a secret helper
nikolas
eemeli
long time no see you should come more often
i have been busy iisakki is no longer young
do you have the list
well i will be so many new children
as a matter of fact one name is missing from that list elsa
is that
a girl three months
let us add her to the list
what is the name of this little princess
aada
aada
hello aada
nikolas meet henrik and hermanni
my sons
i sought them out and asked them here
we were wrong when we
we want to make it up to our father
we came to take him to live with us
to live with you where
away from here father is too old to be living in arctic conditions
you will get the house and the workshop you have earned them
wait nikolas
nikolas
nikolas try to understand

In [140]:
for utt, speaker in tbbt_episode:
    print(utt)
    print(transformation(utt))
    print('=='*50)


if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. if it's unobserved it will, however, if it's observed after it's left the plane but before it hits its target, it will not have gone through both slits.
if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits if it is unobserved it will however if it is observed after it is left the plane but before it hits its target it will not have gone through both slits
agreed, what's your point?
agreed what is your point
there's no point, i just think it's a good idea for a t-shirt.
there is no point i just think it is a good idea for a tshirt
one across is aegean, eight down is nabakov, twenty-six across is mcm, fourteen down is... move your finger... phylum, which makes fourteen across port-au-prince. see, papa doc's capital idea, that's port-au-prince. haiti.
one across is aegean eight down is nabakov twen

In [ ]:
for i in range(start, end):
    if i in subsets:
        print("||||", en_subtitle[i])
        print("||||", zh_subtitle[i])
    else:
        print(en_subtitle[i])
        print(zh_subtitle[i])
    print('=='*50)